In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()
spark

In [5]:
order_items = spark.read.csv("order_items.csv",header = True, inferSchema = True)
order_items.explain(True)

== Parsed Logical Plan ==
Relation [order_id#98,amount#99,profit#100,quantity#101,product_id#102] csv

== Analyzed Logical Plan ==
order_id: string, amount: double, profit: double, quantity: int, product_id: int
Relation [order_id#98,amount#99,profit#100,quantity#101,product_id#102] csv

== Optimized Logical Plan ==
Relation [order_id#98,amount#99,profit#100,quantity#101,product_id#102] csv

== Physical Plan ==
FileScan csv [order_id#98,amount#99,profit#100,quantity#101,product_id#102] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/ShrutiSharma/Downloads/Dataset for PySpark Assignments/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<order_id:string,amount:double,profit:double,quantity:int,product_id:int>



In [18]:
customers = spark.read.csv("customers.csv",header = True, inferSchema = True)
customers.show(5)

+---+-------------+--------------+------+----------+-------------+----------+--------------------+------+
| id| customername|         state|  city|created_on|date_of_birth|updated_on|               email|gender|
+---+-------------+--------------+------+----------+-------------+----------+--------------------+------+
|267|  Mala Pratap|Madhya Pradesh|Indore|2018-12-06|   1983-11-04|2018-12-06|Mala Pratap@outlo...|     f|
| 59|      Anudeep|Madhya Pradesh|Indore|2018-08-26|   1978-09-09|2018-08-26|                NULL|  NULL|
|273|Shakshi Sagar|      Nagaland|Kohima|2018-04-17|   1996-11-06|2019-03-27|     Sagar@gmail.com|     f|
|116| Ekta Chauhan|Madhya Pradesh|Indore|2018-06-28|   1987-04-20|2018-06-28|Ekta Chauhan@outl...|     f|
| 92|     Bhutekar|Madhya Pradesh|Indore|2019-01-04|   1989-10-08|2019-01-04|                NULL|  NULL|
+---+-------------+--------------+------+----------+-------------+----------+--------------------+------+
only showing top 5 rows



In [19]:
orders = spark.read.csv("orders.csv",header = True, inferSchema = True)
orders.show(5)

+-------+----------+-----------+
|     id|order_date|customer_id|
+-------+----------+-----------+
|B-25709|01-07-2018|          1|
|B-26081|22-03-2019|          2|
|B-26018|14-02-2019|          2|
|B-25608|08-04-2018|          2|
|B-25893|04-12-2018|          3|
+-------+----------+-----------+
only showing top 5 rows



In [20]:
product = spark.read.csv("product.csv",header = True, inferSchema = True)
product.show(5)
type(product)

+---+--------+-----------+
| id|category|    product|
+---+--------+-----------+
|  1|Clothing|Hankerchief|
|  2|Clothing|      Kurti|
|  3|Clothing|   Leggings|
|  4|Clothing|      Saree|
|  5|Clothing|      Shirt|
+---+--------+-----------+
only showing top 5 rows



pyspark.sql.dataframe.DataFrame

In [31]:
# 1. Find the total profit generated by each product category
profit = product.join(order_items, product.id == order_items.product_id, "inner")
# print(profit.show(5))
total_profit = profit.groupBy("category").sum("profit")
total_profit.explain(True)

== Parsed Logical Plan ==
'Aggregate ['category], ['category, sum(profit#385) AS sum(profit)#1021]
+- Join Inner, (id#559 = product_id#387)
   :- Relation [id#559,category#560,product#561] csv
   +- Relation [order_id#383,amount#384,profit#385,quantity#386,product_id#387] csv

== Analyzed Logical Plan ==
category: string, sum(profit): double
Aggregate [category#560], [category#560, sum(profit#385) AS sum(profit)#1021]
+- Join Inner, (id#559 = product_id#387)
   :- Relation [id#559,category#560,product#561] csv
   +- Relation [order_id#383,amount#384,profit#385,quantity#386,product_id#387] csv

== Optimized Logical Plan ==
Aggregate [category#560], [category#560, sum(profit#385) AS sum(profit)#1021]
+- Project [category#560, profit#385]
   +- Join Inner, (id#559 = product_id#387)
      :- Project [id#559, category#560]
      :  +- Filter isnotnull(id#559)
      :     +- Relation [id#559,category#560,product#561] csv
      +- Project [profit#385, product_id#387]
         +- Filter isnotn

In [32]:
# 2. Find the number of orders placed by customers in each state  
customer_orders = orders.join(customers, orders.customer_id == customers.id, "inner")
# print(customer_orders.show(5))
orders_in_each_state =  customer_orders.groupBy("state").count()
orders_in_each_state.show()

+-----------------+-----+
|            state|count|
+-----------------+-----+
|         Nagaland|   15|
|        Karnataka|   21|
|       Tamil Nadu|    8|
|   Andhra Pradesh|   15|
|   Madhya Pradesh|  101|
|           Punjab|   25|
|              Goa|   14|
| Himachal Pradesh|   14|
|          Haryana|   14|
|Jammu and Kashmir|   14|
|          Gujarat|   27|
|           Sikkim|   12|
|            Delhi|   22|
|        Rajasthan|   32|
|          Kerala |   16|
|      Maharashtra|   90|
|      West Bengal|   22|
|            Bihar|   16|
|    Uttar Pradesh|   22|
+-----------------+-----+



In [33]:
# 3. Calculate the total amount spent by each customer
customer_orders = order_items.join(orders, order_items.order_id == orders.id, "inner")
# print(customer_orders.show())
total_amount_spent_by_customer = customer_orders.groupBy("customer_id") .sum("amount")
total_amount_spent_by_customer = total_amount_spent_by_customer.join(customers, total_amount_spent_by_customer.customer_id == customers.id, "inner").select(customers["customername"],customers["id"],total_amount_spent_by_customer["sum(amount)"])
total_amount_spent_by_customer.show()

+--------------------+---+-----------+
|        customername| id|sum(amount)|
+--------------------+---+-----------+
|       Kushal Sharma|148|      336.0|
|      Vinit Katariya|392|     2139.0|
|              Pranav|243|     1385.0|
|        Akshay Kumar| 31|      112.0|
|Priyanka Bagcdani...|251|     1752.0|
|         Bhaggyasree| 85|      715.0|
|          Inderpreet|137|       48.0|
|             Arindam| 65|       34.0|
|Ankita Kanwar Rat...| 53|      434.0|
|      Reeta @ Rachna|255|       47.0|
|      Shrimati Seema|296|     5228.0|
|        Hemanti Aary|133|     1643.0|
|              Shruti|322|      970.0|
|  Atul Kumar @ Rahul| 78|     1034.0|
|           Shrichand|321|     3828.0|
|Reeta  And Her Do...|362|       58.0|
|        Tanvir Ahmad|375|      934.0|
|            Kartikay|155|      890.0|
|           Km Dipali|108|       57.0|
|             Swapnil|368|     3395.0|
+--------------------+---+-----------+
only showing top 20 rows



In [34]:
# 4. Find the average profit per order for each city
orders_customers = orders.join(customers, orders.customer_id == customers.id).select(orders["id"].alias("order_id"), customers["id"].alias("customer_id"), customers["city"])
# print(orders_customers.show())
city_order_details = orders_customers.join(order_items, orders_customers.order_id == order_items.order_id)
average_profit_per_order_by_city = city_order_details.groupBy("city").avg("profit").alias("average_profit_per_order")
average_profit_per_order_by_city.show()

+------------------+-------------------+
|              city|        avg(profit)|
+------------------+-------------------+
|         Bangalore| 11.944444444444445|
|           Udaipur|               67.0|
|             Patna|  -5.17741935483871|
|           Gangtok| 16.708333333333332|
|           Chennai|             -88.64|
|           Lucknow|  4.105263157894737|
|            Mumbai|  7.908212560386473|
|               Goa|  8.604651162790697|
|         Ahmedabad|-14.193548387096774|
|           Kolkata| 39.682539682539684|
|         Allahabad|              102.7|
|          Amritsar| 36.266666666666666|
|             Simla| 22.620689655172413|
|             Surat|               53.8|
|           Kashmir|0.16326530612244897|
|              Pune|  54.68674698795181|
|             Delhi|  43.30864197530864|
|        Chandigarh| 2.4225352112676055|
|            Bhopal| 13.196969696969697|
|Thiruvananthapuram| 41.577777777777776|
+------------------+-------------------+
only showing top

In [35]:
# 5. Find the top 5 customers who spent the most
top_5_customers = total_amount_spent_by_customer.orderBy(["sum(amount)"],ascending = [False]).limit(5)
top_5_customers.show()

+--------------------+---+-----------+
|        customername| id|sum(amount)|
+--------------------+---+-----------+
|              Yaanvi|397|     9177.0|
|Reeta  And Her Do...|360|     6611.0|
|              Soumya|343|     6339.0|
|             Harshal|129|     6026.0|
| Smt. Pooja Urf Rani|239|     5809.0|
+--------------------+---+-----------+



In [26]:
# 6. Find the total revenue generated by each product
total_revenue = profit.groupBy("product").sum("amount").show()

+----------------+-----------+
|         product|sum(amount)|
+----------------+-----------+
|         T-shirt|     7382.0|
|           Kurti|     3361.0|
|          Chairs|    34222.0|
|     Furnishings|    13484.0|
|     Hankerchief|    14608.0|
|       Bookcases|    56861.0|
|           Saree|    53511.0|
|        Leggings|     2106.0|
|           Stole|    18546.0|
|     Accessories|    21728.0|
|          Phones|    46119.0|
|        Printers|    58252.0|
|        Trousers|    30039.0|
|           Shirt|     7555.0|
|          Tables|    22614.0|
|           Skirt|     1946.0|
|Electronic Games|    39168.0|
+----------------+-----------+



In [27]:
# 7. Find the number of orders placed in each product category
number_of_orders_categorywise = profit.groupBy("category").count()
number_of_orders_categorywise.show()

+-----------+-----+
|   category|count|
+-----------+-----+
|Electronics|  308|
|   Clothing|  949|
|  Furniture|  243|
+-----------+-----+



In [28]:
# 8. Calculate the average profit per order for each product category
avg_profit = profit.groupBy("category").avg("profit")
avg_profit.show()

+-----------+-----------------+
|   category|      avg(profit)|
+-----------+-----------------+
|Electronics|34.07142857142857|
|   Clothing|11.76290832455216|
|  Furniture| 9.45679012345679|
+-----------+-----------------+



In [29]:
# 9. Find the total amount spent in each city.
amount_in_each_city = city_order_details.groupBy("city").sum("amount")
amount_in_each_city.show()

+------------------+-----------+
|              city|sum(amount)|
+------------------+-----------+
|         Bangalore|    15058.0|
|           Udaipur|    11073.0|
|             Patna|    12943.0|
|           Gangtok|     5276.0|
|           Chennai|     6087.0|
|           Lucknow|     5502.0|
|            Mumbai|    61867.0|
|               Goa|     6705.0|
|         Ahmedabad|    14230.0|
|           Kolkata|    14086.0|
|         Allahabad|    16857.0|
|          Amritsar|     4507.0|
|             Simla|     8666.0|
|             Surat|     6828.0|
|           Kashmir|    10829.0|
|              Pune|    33481.0|
|             Delhi|    25019.0|
|        Chandigarh|    21142.0|
|            Bhopal|    23583.0|
|Thiruvananthapuram|    13459.0|
+------------------+-----------+
only showing top 20 rows



In [30]:
# 10. Find the number of orders placed for each product. 
number_of_orders = profit.groupBy("product").count()
number_of_orders.show()

+----------------+-----+
|         product|count|
+----------------+-----+
|         T-shirt|   77|
|           Kurti|   47|
|          Chairs|   74|
|     Furnishings|   73|
|     Hankerchief|  198|
|       Bookcases|   79|
|           Saree|  210|
|        Leggings|   53|
|           Stole|  192|
|     Accessories|   72|
|          Phones|   83|
|        Printers|   74|
|        Trousers|   39|
|           Shirt|   69|
|          Tables|   17|
|           Skirt|   64|
|Electronic Games|   79|
+----------------+-----+

